In [ ]:
from qa_qc_lib.qa_qc_seismic import QA_QC_seismic

In [ ]:
file_path = 'path to file'
data1_tests = QA_QC_seismic(file_path=file_path)

In [ ]:
list_of_tests = data1_tests.get_list_of_tests()
list_of_tests

In [ ]:
data1_tests.start_tests(list_of_tests[0:5])

In [ ]:
data1_tests.generate_test_report()